## bifacial_radiance examples
what to do with the new bifacial_radiance python module

### Prerequisites:
This software requires the previous installation of RADIANCE from https://github.com/NREL/Radiance/releases

If you are on a PC you should also install the Jaloxa radwinexe executables: http://www.jaloxa.eu/resources/radiance/radwinexe.shtml

#### STEP 1: Install and import the program

 - clone the repo to your local directory
 - navigate to the \bifacial_radiance directory which contains setup
 - run `pip install -e .  `

Gencumulativesky requires gencumulativesky.exe to be installed in your Radiance directory.  Look in the /bifacial_radiance/data/ directory and copy gencumulativesky.exe to your Radiance install directory.

In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')

In [2]:
# Simple example system using Radiance.
#point to an empty directory or existing Radiance directory
testfolder = r'C:\Users\cdeline\Documents\Python Scripts\TestFolder'  
demo = RadianceObj('bifacial_example',testfolder)  # Create a RadianceObj 'object' named bifacial_example. no whitespace allowed

path = C:\Users\cdeline\Documents\Python Scripts\TestFolder


In [3]:
demo.setGround(0.62) # input albedo number or material name like 'concrete'.  To see options, run this without any input.

# pull in meteorological data using pyEPW for any global lat/lon
epwfile = demo.getEPW(37.5,-77.6) 

# read in the weather data pulled in above. 
# If you want a different location, replace this filename with the new EPW file name in `epwfile`.    
metdata = demo.readEPW('EPWs\\USA_VA_Richmond.Intl.AP.724010_TMY.epw') 

# Solar resource definition.  Either choose a single time point, or use cumulativesky for the entire year. 
fullYear = True
if fullYear:
    demo.genCumSky(demo.epwfile) # entire year.
    # NOTE: this will claim to throw an 'error' if there are times when the sun is below the horizon.  This is normal!
else:
    demo.gendaylit(metdata,4020)  # Noon, June 17th (timepoint # 4020)


Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
error: There were 4255 sun up hours in this climate file
Total Ibh/Lbh: 0.000000


In [4]:
# create a custom PV module type. Prism Solar Bi60. x = .984 y = 1.695. Bifaciality = 0.90
# Note: modules are currently 100% opaque.  This will be modified in the future
demo.makeModule(name='Prism Solar Bi60',x=0.984,y=1.695,bifi = 0.90)

# print available module types in data/module.json
demo.printModules()


Module Prism Solar Bi60 successfully created
Available module names: ['mini_panel', 'simple_panel', 'monopanel', 'Prism Solar Bi60']


In [ ]:
# create a scene using panels in landscape at 10 deg tilt, 1.5m pitch. 0.2 m ground clearance
sceneDict = {'tilt':10,'pitch':1.5,'height':0.2,'orientation':'landscape','azimuth':180} 
module_type = 'monopanel'
scene = demo.makeScene(module_type,sceneDict, nMods = 20, nRows = 7) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.
analysis = AnalysisObj(octfile, demo.basename)  # return an analysis object including the scan dimensions for back irradiance
analysis.analysis(octfile, demo.basename, scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
# the frontscan and backscan include a linescan along a chord of the module, both on the front and back.  
# Return the minimum of the irradiance values, and the average of the irradiance values along a chord of the module.
print('Annual bifacial ratio: %0.3f - %0.3f' %(min(analysis.backRatio), np.mean(analysis.backRatio)) )



In [ ]:
# Make a color render and falsecolor image of the scene
analysis.makeImage('side.vp')
analysis.makeFalseColor('side.vp')

In [ ]:
# some other older examples.  Backward compatibility not guaranteed!
'''
    demo = RadianceObj('G173gencumsky_3.0m')  
    demo.setGround('litesoil')
    metdata = demo.readEPW(r'USA_CO_Boulder.724699_TMY2.epw')
    # sky data for index 4010 - 4028 (June 17)  
    #demo.gendaylit(metdata,4020)
    demo.genCumSky(r'USA_CO_Boulder.724699_TMY2.epw')
    octfile = demo.makeOct(demo.filelist + ['objects\\monopanel_G173_ht_3.0.rad'])
    analysis = AnalysisObj(octfile, demo.basename)
    analysis.G173analysis(octfile, demo.basename)
    
    demo2 = RadianceObj('G173gendaylit_3.0m')  
    demo2.setGround('litesoil')
    metdata = demo2.readEPW(r'USA_CO_Boulder.724699_TMY2.epw')
    # sky data for index 4010 - 4028 (June 17)  
    demo2.gendaylit(metdata,4020)
    #demo.genCumSky(r'USA_CO_Boulder.724699_TMY2.epw')
    octfile = demo2.makeOct(demo.filelist + ['objects\\monopanel_G173_ht_3.0.rad'])
    analysis2 = AnalysisObj(octfile, demo2.basename)
    analysis2.G173analysis(octfile, demo2.basename)

    pvscdemo = RadianceObj('PVSC_gencumsky')  
    pvscdemo.setGround('litesoil')
    epwfile = pvscdemo.getEPW(40,-105)
    metdata = pvscdemo.readEPW(epwfile)
    # sky data for index 4010 - 4028 (June 17)  
    #demo.gendaylit(metdata,4020)
    start = datetime.datetime(2000,6,17,12)
    end = datetime.datetime(2000,6,17,13)
    pvscdemo.genCumSky(r'USA_CO_Boulder.724699_TMY2.epw', start, end)
    octfile = pvscdemo.makeOct(pvscdemo.filelist + ['objects\\PVSC_4array.rad'])
    pvscdemo.analysis(octfile, pvscdemo.basename)

    pvscdemo = RadianceObj('PVSC_gendaylit')  
    pvscdemo.setGround('litesoil')
    metdata = pvscdemo.readEPW(r'USA_CO_Boulder.724699_TMY2.epw')
    # sky data for index 4010 - 4028 (June 17)  
    pvscdemo.gendaylit(metdata,4020)
    #pvscdemo.genCumSky(r'USA_CO_Boulder.724699_TMY2.epw',datetime.datetime(2000,6,17,0), datetime.datetime(2000,6,17,23))
    octfile = pvscdemo.makeOct(pvscdemo.filelist + ['objects\\PVSC_4array.rad'])
    analysis = pvscdemo.analysis(octfile, pvscdemo.basename)
    analysis.makeImage('PVSCfront.vp')
    '''